# Scikit-Learn SVM Pipeline Testing

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

wl = wallaroo.Client(auth_type="sso", interactive=True)

## Data & Model Creation

In [2]:
data = load_iris(as_frame=True)

X = data['data'].values
y = data['target']

data['data'].iloc[0:2].to_json('data/test_svm_pipeline_data.json', orient='records')

In [3]:
input_schema = pa.Schema.from_pandas(data['data'])
output_schema = pa.schema([
    pa.field('output', pa.float64())
])

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
pipe.fit(X, y)

In [6]:
pickle.dump(pipe, open('models/svm_pipeline.pkl', 'wb'))

## Upload model

In [ ]:
model = wl.upload_model('pipe-test', 'models/svm_pipeline.pkl', framework=Framework.SKLEARN, input_schema=input_schema, output_schema=output_schema)
model

## Configure model and pipeline

In [10]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [ ]:
pipeline_name = f"svm-pipeline"
pipeline = wl.build_pipeline(pipeline_name)
pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

## Inference

In [ ]:
pipeline.infer_from_file('data/test_svm_pipeline_data.json')

In [ ]:
for pipeline in wl.list_pipelines():
    pipeline.undeploy()